## Using Agent Framework with MCP Servers

The Microsoft Agent Framework supports integration with Model Context Protocol (MCP) servers, allowing your agents to access external tools and services. This exercise shows how to connect to an MCP server and use its tools within your agent.

The Python Agent Framework provides comprehensive support for integrating with Model Context Protocol (MCP) servers through multiple connection types. This allows your agents to access external tools and services seamlessly.

Let's start by initialising our `chat_client`

In [ ]:
import os
from dotenv import load_dotenv
from agent_framework.azure import AzureOpenAIChatClient

# Load environment variables
load_dotenv()

endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")
deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

# Create the client using API key
client = AzureOpenAIChatClient(
    endpoint=endpoint,
    api_key=api_key,
    api_version=api_version,
    deployment_name=deployment
)

--- 
Perfect. All setup.

Check the following code snippet to see how can we connect to the Microsoft Learn MCP Server.
Your task is to create an agent that is able to call the MCP server and get up-to-date information of a certain Microsoft product.

In [ ]:
from agent_framework import ChatAgent, MCPStreamableHTTPTool

agent: ChatAgent = client.create_agent(
    # Add your agent parameters and instructions here
)

async with MCPStreamableHTTPTool(
        name="Microsoft Learn MCP",
        url="https://learn.microsoft.com/api/mcp",
        chat_client=client
    ) as mcp_tool:

        result = await agent.run(
            # Add your question and mcp_server here
        )
        print("\n=== Answer ===\n", result.text)

<details>
    <summary> See the solution</summary>

```python
from agent_framework import ChatAgent, MCPStreamableHTTPTool

agent: ChatAgent = client.create_agent(
    name="DocsAgent",
    instructions=(
        "You are a helpful assistant that can help with Microsoft documentation questions."
    ),
)

async with MCPStreamableHTTPTool(
        name="Microsoft Learn MCP",
        url="https://learn.microsoft.com/api/mcp",
        chat_client=client
    ) as mcp_tool:

        result = await agent.run(
            "How to create an Azure storage account using az cli?", 
            tools=mcp_tool
        )
        print("\n=== Answer ===\n", result.text)
```
</details>

---

Nice, we have connected to the Microsoft Learn MCP server. It's useful when GitHub Copilot needs to dig deeper into Microsoft documentation to provide you with actual and relevant information.
However, we are also able to setup our own MCP servers.

In this workshop we have provided an out-of-the-box MCP server, built using FastMCP. To run the MCP server, open your terminal and run the following commands:

- ```cd mcp```
- ```uv run --prerelease=allow mcp_server.py```


The MCP server will start running and output the address like so:

```
INFO      Starting MCP server 'Contoso Customer API as Tools' with transport 'http' on http://127.0.0.1:8000/mcp     
INFO:     Started server process [36000]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
```

Now try to access the local MCP server and output the name of the MCP server.

In [ ]:
from agent_framework import MCPStreamableHTTPTool

async with MCPStreamableHTTPTool()

<details>
<summary>See the solution</summary>

```python
from agent_framework import MCPStreamableHTTPTool

async with MCPStreamableHTTPTool(
    name="ContosoMCP",
    url="http://127.0.0.1:8000/mcp",
    timeout=60,
    chat_client=client
) as contoso_mcp_tool:
    print("MCP tool connected successfully")
    print(f"MCP Tool name: {contoso_mcp_tool.name}")

```
</details>

---

Great, you have connected to the local MCP server.
The next goal is to create a customer service agent that uses the local MCP server to look up customer data.

**Your Task**:
1. Connect to the ContosoMCP server at `http://127.0.0.1:8000/mcp`
2. Create a ChatAgent with customer service instructions
3. Add the MCP server as a tool to the agent
4. Test with: "Look up customer details for customer ID 2"

**Hint**: Use nested `async with` blocks - MCP tool first, then ChatAgent.

---

In [ ]:
from agent_framework import ChatAgent, MCPStreamableHTTPTool

async with MCPStreamableHTTPTool(
    
    ) as contoso_mcp_tool:

<details>
<summary> See the solution</summary>

```python
from agent_framework import ChatAgent, MCPStreamableHTTPTool

async with MCPStreamableHTTPTool(
    name="ContosoMCP",
    url="http://127.0.0.1:8000/mcp",
    timeout=60,
    chat_client=client
) as contoso_mcp_tool:
    
    async with ChatAgent(
        chat_client=client,
        name="TestBot", 
        instructions="You are a customer service agent. Use the available tools to help customers. "
        "Always try to use the customer lookup tools when given a customer ID.",
    ) as agent:
        
        print("Testing simple customer lookup...")
        response = await agent.run(
            messages="Look up customer details for customer ID 2",
            tools=contoso_mcp_tool
        )
        
        print("Response:", response.text)
```

</details>